In [35]:
import os
import sys
import argparse
import re
import time
import random
from datetime import datetime
from typing import Any, Tuple, Dict, List
from copy import deepcopy
import copy
import math
import shutil


from tqdm import tqdm
from sklearn import linear_model, model_selection
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
import torch
from torch.optim.lr_scheduler import _LRScheduler
from torch.nn import functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset, Subset, ConcatDataset, dataset
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100, CIFAR10, ImageFolder
from torchvision.models import resnet18
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from transformers import ViTModel, ViTFeatureExtractor
import seaborn as sns
import scipy.stats as stats
import pandas as pd
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from transformers.data.processors import SingleSentenceClassificationProcessor, InputFeatures
from transformers import AutoModel, AutoTokenizer , AutoModelForSequenceClassification, AutoConfig


# Original code from https://github.com/weiaicunzai/pytorch-cifar100 <- refer to this repo for comments

In [36]:
DATE_FORMAT = "%A_%d_%B_%Y_%Hh_%Mm_%Ss"

# time of script run
TIME_NOW = datetime.now().strftime(DATE_FORMAT)

In [37]:
class HARD():
    def __init__(self, train, file_path):
        df = pd.read_csv(file_path)
        if(train):
            df = df.loc[:40000]
        else:
            df = df.loc[40000:]
        
        self.data = df["review"].tolist()
        self.targets = df["rating"].tolist()
        model_dir = "/kaggle/input/marbert-hard-data"
        
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        
        dataset = SingleSentenceClassificationProcessor(mode='classification')
        dataset.add_examples(texts_or_text_and_labels=self.data,overwrite_examples = True)
        
        tokenizer.max_len = 512
        self.data = dataset.get_features(tokenizer = tokenizer, max_length =512)
    
            
    def __getitem__(self, index):
        review = self.data[index]
        review_dict = {"input_ids":torch.tensor(review.input_ids), "attention_mask": torch.tensor(review.attention_mask)}
        return (review_dict,self.targets[index])
    
    def __len__(self):
        # Assuming 'data' is a list attribute, return its length
        return len(self.data)

In [38]:
class Marbert(nn.Module):
    def __init__(self, num_classes=5, model_dir="", **kwargs):
        super(Marbert, self).__init__()
       
        config = AutoConfig.from_pretrained(model_dir, num_labels=num_classes)
        
        self.base = AutoModelForSequenceClassification.from_pretrained(model_dir, config = config)
        self.num_classes = num_classes
        self.config = config
    
    def forward(self, input_ids, attention_mask):
        # Forward pass computation
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

    def __call__(self, reviews):
        return super(Marbert, self).__call__(reviews['input_ids'].to(torch.int64).to("cuda"), reviews['attention_mask'].to(torch.int64).to("cuda")).to("cuda")

In [39]:
datasets_dict = {
    "HARD": HARD
}

In [40]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
import numpy as np

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)) * 100


def training_step(model, batch, device):
    review, rating, _ = batch
    out = model(review)  # Generate predictions
    loss = F.cross_entropy(out, rating)  # Calculate loss
    return loss


def validation_step(model, batch, device):
    review, rating = batch
    rating = rating.to("cuda")
    out = model(review)  # Generate predictions
    loss = F.cross_entropy(out, rating)  # Calculate loss
    acc = accuracy(out, rating)  # Calculate accuracy
    return {"Loss": loss.detach(), "Acc": acc}


def validation_epoch_end(model, outputs):
    batch_losses = [x["Loss"] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
    batch_accs = [x["Acc"] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()  # Combine accuracies
    return {"Loss": epoch_loss.item(), "Acc": epoch_acc.item()}


def epoch_end(model, epoch, result):
    print(
        "Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch,
            result["lrs"][-1],
            result["train_loss"],
            result["Loss"],
            result["Acc"],
        )
    )


@torch.no_grad()
def evaluate(model, val_loader, device):
    model.eval()
    outputs = [validation_step(model, batch, device) for batch in val_loader]
    return validation_epoch_end(model, outputs)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

def fit_one_unlearning_cycle(epochs, model, train_loader, val_loader, lr, device):
    history = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = training_step(model, batch, device)
            loss.backward()
            train_losses.append(loss.detach().cpu())

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))

        result = evaluate(model, val_loader, device)
        result["train_loss"] = torch.stack(train_losses).mean()
        result["lrs"] = lrs
        epoch_end(model, epoch, result)
        history.append(result)
    return history

class UNSIR_noise(torch.nn.Module):
    def __init__(self, *dim):
        super().__init__()
        # marbert tokenizer vocab size is 100000
        rand_noise = 1 + (100000 - 1) * torch.rand(*dim)
        self.noise = torch.nn.Parameter(rand_noise, requires_grad=True)

    def forward(self):
        return self.noise


def UNSIR_noise_train(
    noise, model, forget_class_label, num_epochs, noise_batch_size, device="cuda"
):
    opt = torch.optim.Adam(noise.parameters(), lr=0.1)
    model.to(device)  # Ensure the model is on the correct device
    noise.to(device)  # Ensure the noise parameters are on the correct device

    for epoch in range(num_epochs):
        inputs = noise()
        inputs = inputs.to(device)
        attention_mask = torch.ones(inputs.shape).to(device)
        
        dict_inputs = {'input_ids': inputs, 'attention_mask': attention_mask}
        labels = torch.full((noise_batch_size,), forget_class_label, device=device, dtype=torch.long)
        
        outputs = model(dict_inputs)
        loss = -F.cross_entropy(outputs, labels) + 0.1 * torch.mean(
                torch.sum(inputs**2, dim=1)
        )
        opt.zero_grad()
        loss.backward()
        opt.step()

        total_loss = loss.item()
        
        if epoch % 4 == 0:
            print(f"Epoch {epoch} Loss: {total_loss}")

    return noise


def UNSIR_create_noisy_loader(
    noise,
    forget_class_label,
    retain_samples,
    batch_size,
    num_noise_batches=80,
    device="cuda",
):
    noisy_data = []
    for i in range(num_noise_batches):
        batch = noise()
        for i in range(batch.size(0)):
            dict_inputs = {'input_ids': batch[i].detach().cpu(),'attention_mask':torch.ones(batch[i].shape).detach().cpu()}
            noisy_data.append(
                (
                    dict_inputs,
                    torch.tensor(forget_class_label).to(device),
                    torch.tensor(forget_class_label).to(device),
                )
            )
    other_samples = []
    for i in range(len(retain_samples)):
        dict_inputs = {'input_ids': retain_samples[i][0]['input_ids'].cpu(),
                        'attention_mask':retain_samples[i][0]['attention_mask'].cpu()}
        other_samples.append(
            (
                dict_inputs,
                torch.tensor(retain_samples[i][1]).to(device),
                torch.tensor(retain_samples[i][1]).to(device),
            )
        )
    noisy_data += other_samples
    noisy_loader = DataLoader(noisy_data, batch_size=batch_size, shuffle=True)

    return noisy_loader

In [41]:
"""
From https://github.com/vikram2000b/bad-teaching-unlearning / https://arxiv.org/abs/2205.08096
"""

def JSDiv(p, q):
    m = (p + q) / 2
    return 0.5 * F.kl_div(torch.log(p), m) + 0.5 * F.kl_div(torch.log(q), m)


# ZRF/UnLearningScore https://arxiv.org/abs/2205.08096
def UnLearningScore(tmodel, gold_model, forget_dl, batch_size, device):
    model_preds = []
    gold_model_preds = []
    with torch.no_grad():
        for batch in forget_dl:
            x, y = batch
            x = {'input_ids': x['input_ids'].to(device),
                    'attention_mask':x['attention_mask'].to(device)}
            model_output = tmodel(x)
            gold_model_output = gold_model(x)
            model_preds.append(F.softmax(model_output, dim=1).detach().cpu())
            gold_model_preds.append(F.softmax(gold_model_output, dim=1).detach().cpu())

    model_preds = torch.cat(model_preds, axis=0)
    gold_model_preds = torch.cat(gold_model_preds, axis=0)
    return 1 - JSDiv(model_preds, gold_model_preds)


def entropy(p, dim=-1, keepdim=False):
    return -torch.where(p > 0, p * p.log(), p.new([0.0])).sum(dim=dim, keepdim=keepdim)


def collect_prob(data_loader, model):
    data_loader = torch.utils.data.DataLoader(
        data_loader.dataset, batch_size=1, shuffle=False
    )
    prob = []
    with torch.no_grad():
        for batch in data_loader:
            data,target = batch
            output = model(data)
            prob.append(F.softmax(output, dim=-1).data)
    return torch.cat(prob)


# https://arxiv.org/abs/2205.08096
def get_membership_attack_data(retain_loader, forget_loader, test_loader, model):
    retain_prob = collect_prob(retain_loader, model)
    forget_prob = collect_prob(forget_loader, model)
    test_prob = collect_prob(test_loader, model)

    X_r = (
        torch.cat([entropy(retain_prob), entropy(test_prob)])
        .cpu()
        .numpy()
        .reshape(-1, 1)
    )
    Y_r = np.concatenate([np.ones(len(retain_prob)), np.zeros(len(test_prob))])

    X_f = entropy(forget_prob).cpu().numpy().reshape(-1, 1)
    Y_f = np.concatenate([np.ones(len(forget_prob))])
    return X_f, Y_f, X_r, Y_r


# https://arxiv.org/abs/2205.08096
def get_membership_attack_prob(retain_loader, forget_loader, test_loader, model):
    X_f, Y_f, X_r, Y_r = get_membership_attack_data(
        retain_loader, forget_loader, test_loader, model
    )
    # clf = SVC(C=3,gamma='auto',kernel='rbf')
    clf = LogisticRegression(
        class_weight="balanced", solver="lbfgs", multi_class="multinomial"
    )
    clf.fit(X_r, Y_r)
    results = clf.predict(X_f)
    return results.mean()


@torch.no_grad()
def actv_dist(model1, model2, dataloader, device="cuda"):
    print("actv_dist")
    sftmx = nn.Softmax(dim=1)
    distances = []
    for batch in dataloader:
        x, _, _ = batch
        x = x.to(device)
        model1_out = model1(x)
        model2_out = model2(x)
        diff = torch.sqrt(
            torch.sum(
                torch.square(
                    F.softmax(model1_out, dim=1) - F.softmax(model2_out, dim=1)
                ),
                axis=1,
            )
        )
        diff = diff.detach().cpu()
        distances.append(diff)
    distances = torch.cat(distances, axis=0)
    return distances.mean()

In [42]:
# Returns metrics
def get_metric_scores(
    model,
    unlearning_teacher,
    retain_train_dl,
    retain_valid_dl,
    forget_train_dl,
    forget_valid_dl,
    valid_dl,
    device,
):
    loss_acc_dict = evaluate(model, valid_dl, device)
    retain_acc_dict = evaluate(model, retain_valid_dl, device)
    zrf = UnLearningScore(model, unlearning_teacher, forget_valid_dl, 128, device)
    d_f = evaluate(model, forget_valid_dl, device)
    mia = get_membership_attack_prob(retain_train_dl, forget_train_dl, valid_dl, model)

    return (loss_acc_dict["Acc"], retain_acc_dict["Acc"], zrf, mia, d_f["Acc"])


# Implementation from https://github.com/vikram2000b/Fast-Machine-Unlearning
def UNSIR(
    model,
    unlearning_teacher,
    retain_train_dl,
    retain_valid_dl,
    forget_train_dl,
    forget_valid_dl,
    valid_dl,
    num_classes,
    forget_class,
    device,
    **kwargs,
):
    
    classwise_train = get_classwise_ds_marbert(
        ConcatDataset((retain_train_dl.dataset, forget_train_dl.dataset)), num_classes, "full"
    )
    noise_batch_size = 8
    retain_valid_dl = DataLoader(retain_valid_dl.dataset, batch_size=noise_batch_size)
    # collect some samples from each class
    num_samples = 500
    retain_samples = []
    for i in range(num_classes):
        if i != forget_class:
            retain_samples += classwise_train[i][:num_samples]

    forget_class_label = forget_class
#     img_shape = next(iter(retain_train_dl.dataset))[0].shape[-1]
    noise = UNSIR_noise(noise_batch_size, 512).to(device)
    noise = UNSIR_noise_train(
        noise, model, forget_class_label, 25, noise_batch_size, device=device
    )
    noisy_loader = UNSIR_create_noisy_loader(
        noise,
        forget_class_label,
        retain_samples,
        batch_size=noise_batch_size,
        device=device,
    )
    # impair step
    _ = fit_one_unlearning_cycle(
        1, model, noisy_loader, retain_valid_dl, device=device, lr=0.0001
    )
    
    # repair step
    other_samples = []
    for i in range(len(retain_samples)):
        dict_inputs = {'input_ids': retain_samples[i][0]['input_ids'].to(device),
                    'attention_mask':retain_samples[i][0]['attention_mask'].to(device)}
        other_samples.append(
            (
                dict_inputs,
                torch.tensor(retain_samples[i][1]).to(device),
                torch.tensor(retain_samples[i][1]).to(device),
            )
        )

    heal_loader = torch.utils.data.DataLoader(
        other_samples, batch_size=8, shuffle=True
    )
    _ = fit_one_unlearning_cycle(
        1, model, heal_loader, retain_valid_dl, device=device, lr=0.0001
    )
    
    return get_metric_scores(
        model,
        unlearning_teacher,
        retain_train_dl,
        retain_valid_dl,
        forget_train_dl,
        forget_valid_dl,
        valid_dl,
        device,
    )



In [43]:
def get_classwise_ds_marbert(ds, num_classes, forget_type):
    classwise_ds = {}
    for i in range(num_classes):
        classwise_ds[i] = []

    for review,rating in ds:
        classwise_ds[rating].append((review,rating))
    return classwise_ds

# Creates datasets for method execution
def build_retain_forget_sets_marbert(
    classwise_train, classwise_test, num_classes, forget_class
):
    # Getting the forget and retain validation data
    forget_valid = []
    for cls in range(num_classes):
        if cls == forget_class:
            for review,rating in classwise_test[cls]:
                forget_valid.append((review,rating))

    retain_valid = []
    for cls in range(num_classes):
        if cls != forget_class:
            for review,rating in classwise_test[cls]:
                retain_valid.append((review,rating))

    forget_train = []
    for cls in range(num_classes):
        if cls == forget_class:
            for review,rating in classwise_train[cls]:
                forget_train.append((review,rating))

    retain_train = []
    for cls in range(num_classes):
        if cls != forget_class:
            for review,rating in classwise_train[cls]:
                retain_train.append((review,rating))
    return (retain_train, retain_valid, forget_train, forget_valid)

# Full Class

In [ ]:
for f_class in [1, 2, 3, 4, 5]:
    print(str(f_class) + " :-")

    """
    This file is used to collect all arguments for the experiment, prepare the dataloaders, call the method for forgetting, and gather/log the metrics.
    Methods are executed in the strategies file.
    """
    res = []
    """
    Get Args
    """
    args = {
        "net": Marbert,
        "weight_path": "/kaggle/working/ResNet18-Cifar100-197-best.pth",
        #choices=["Cifar10", "Cifar20", "Cifar100", "PinsFaceRecognition", "HARD"]
        "dataset": "HARD",
    "classes": 5,
    "gpu": True,
    "b": 32,
    "warm": 1,
    "lr": 0.1,
    "method": UNSIR,
    "forget_class": f_class,
    "epochs": 1,
    "seed": 0
    }

    # Set seeds
    torch.manual_seed(args["seed"])
    np.random.seed(args["seed"])
    random.seed(args["seed"])


    forget_class = args["forget_class"]-1

    batch_size = args["b"]


    # get network
    net = args["net"](num_classes=args["classes"],model_dir="/kaggle/input/marbert-hard-data")

    # for bad teacher
    unlearning_teacher = args["net"](num_classes=args["classes"],model_dir = "/kaggle/input/marbert-hard-data")

    if args["gpu"]:
        net = net.cuda()
        unlearning_teacher = unlearning_teacher.cuda()

    trainset = datasets_dict[args["dataset"]](
        train=True, file_path="/kaggle/input/marbert-hard-data/HARD_50000_clean_shifted.csv"
    )
    validset = datasets_dict[args["dataset"]](
        train=False, file_path="/kaggle/input/marbert-hard-data/HARD_50000_clean_shifted.csv"
    )

    # Set up the dataloaders and prepare the datasets
    trainloader = DataLoader(trainset, num_workers=4, batch_size=args["b"], shuffle=True)
    validloader = DataLoader(validset, num_workers=4, batch_size=args["b"], shuffle=False)

    classwise_train, classwise_test = get_classwise_ds_marbert(
        trainset, args["classes"],"full"
    ), get_classwise_ds_marbert(validset, args["classes"],"full")

    (
        retain_train,
        retain_valid,
        forget_train,
        forget_valid,
    ) = build_retain_forget_sets_marbert(
        classwise_train, classwise_test, args["classes"], forget_class
    )
    forget_valid_dl = DataLoader(forget_valid, batch_size)
    retain_valid_dl = DataLoader(retain_valid, batch_size)

    forget_train_dl = DataLoader(forget_train, batch_size)
    retain_train_dl = DataLoader(retain_train, batch_size, shuffle=True)
    full_train_dl = DataLoader(
        ConcatDataset((retain_train_dl.dataset, forget_train_dl.dataset)),
        batch_size=batch_size,
    )

    kwargs = {
        "model": net,
        "unlearning_teacher": unlearning_teacher,
        "retain_train_dl": retain_train_dl,
        "retain_valid_dl": retain_valid_dl,
        "forget_train_dl": forget_train_dl,
        "forget_valid_dl": forget_valid_dl,
        "full_train_dl": full_train_dl,
        "valid_dl": validloader,
        "forget_class": forget_class,
        "num_classes": args["classes"],
        "dataset_name": args["dataset"],
        "device": "cuda" if args["gpu"] else "cpu",
        "model_name": args["net"],
    }

    start = time.time()

    # executes the method passed via args
    testacc, retainacc, zrf, mia, d_f = args["method"](
        **kwargs
    )
    end = time.time()
    time_elapsed = end - start

    # Logging
    res_dict = {
            "TestAcc": testacc,
            "RetainTestAcc": retainacc,
            "Df": d_f,
            "ZRF": zrf,
            "MIA": mia,
            "model_scaler": model_size_scaler,
            "MethodTime": time_elapsed,  # do not forget to deduct baseline time from it to remove results calc (acc, MIA, ...)
        }

    for k,v in res_dict.items():
        print(k + ": " + str(v))